In [82]:
import requests
import re
import time
import math
import threading
import logging
import pandas as pd

from bs4 import BeautifulSoup
#from IPython.display import clear_output
from tqdm.notebook import trange, tqdm

import csv
import json
#import pprint

In [83]:
url_list = []

with open('links.csv', 'r', newline='') as file:
    rows = csv.reader(file)
    
    for row in rows:
        url_list.append(row[0])

url_list.pop(0)
        
print(len(url_list))
print(url_list[0])

8297
https://distiller.com/spirits/wheatley-vodka


### User Reviews Crawler

In [95]:
all_reviews = []

In [103]:
#User Reviews

def getUserReviews(url_list=url_list, start=0, end=len(url_list)):   
    
    global all_reviews
    
    user_reviews = []
    
    global exec_count 
    
    for url in tqdm(url_list[start:end]):   
        
        html = requests.get(f'{url}/tastes/?page=1')
        bsObj = BeautifulSoup(html.text)
        
        try:
            page_count = math.ceil(int(bsObj.find('span', 
            {'class':'pagination-control__description'}).string.strip().split(' ')[3]) / 10)
            
        except: page_count = 1
    
        for page in range(1, page_count+1):
            
            html = requests.get(f'{url}/tastes/?page={page}')
            bsObj = BeautifulSoup(html.text)       
                         
            try:          
                                
                #product
                product = bsObj.find('h1', {'itemprop':'name'})    
                
                #user
                try:
                    users = bsObj.find_all('h3', 
                                                {'class':'mini-headline name username truncate-line'})
                except:    
                    pass

                #user rating
                try:
                    user_ratings = bsObj.find_all('div', 
                                                {'class':'rating-display__value'})
                except: 
                    pass
                    
                #user comment
                try:
                    user_comments = bsObj.find_all('div',
                                                    {'class':'body'})
                except:
                    pass
                
                for index, user in enumerate(users):
                    
                    user_review = {
                        'product' : None,
                        'user' : None,
                        'user_rating' : None,
                        'user_comment': None
                        }     
                    
                    try: user_review['product'] = product.string.strip()
                    except: pass
                    try: user_review['user'] = user.string.strip()
                    except: pass
                    try: user_review['user_rating'] = float(user_ratings[index].string)
                    except: pass    
                    try: user_review['user_comment'] = re.sub('[\r\n]', '', user_comments[index].string.strip())
                    except: pass
                    
                    #Add to a list   
                    user_reviews.append(user_review)

            except Exception as e:
                
                logging.exception(e)
                print(e)
                time.sleep(5)
                
                continue
        
        time.sleep(1)

    return all_reviews.extend(user_reviews)

In [104]:
getUserReviews(url_list, 0, 1)

In [105]:
print(len(all_reviews))

for i in all_reviews:
    print(i)

70
{'product': 'Wheatley Vodka', 'user': 'MajorHop', 'user_rating': 3.75, 'user_comment': 'Pass. No flavor at all. Rubbing alcohol. I guess it would be work for mixing.'}
{'product': 'Wheatley Vodka', 'user': 'Greonidas', 'user_rating': 4.5, 'user_comment': 'I liked this vodka from the first sip. I like it neat, on the rocks with a few olives.Wheat vodkas are my preference and this one delivers a smooth mellow taste thatI enjoy immensely. This is definitely  a vodka I will purchase again.'}
{'product': 'Wheatley Vodka', 'user': 'jtanderon6', 'user_rating': 4.0, 'user_comment': 'Smooth'}
{'product': 'Wheatley Vodka', 'user': 'Russell-WJ', 'user_rating': 4.0, 'user_comment': None}
{'product': 'Wheatley Vodka', 'user': 'givemesomemore', 'user_rating': 2.0, 'user_comment': None}
{'product': 'Wheatley Vodka', 'user': 'aphelps11', 'user_rating': 4.5, 'user_comment': None}
{'product': 'Wheatley Vodka', 'user': 'Allan-Swartz', 'user_rating': 5.0, 'user_comment': None}
{'product': 'Wheatley Vod

In [107]:
#Dividing the urls into segments

seg = 10
seg_list=[]

for i in range(seg):
    seg_list.append(int(len(url_list)/seg*i))

seg_list.append(len(url_list))

print(seg_list)
print(len(seg_list))

[0, 829, 1659, 2489, 3318, 4148, 4978, 5807, 6637, 7467, 8297]
11


In [110]:
threads = []
num = len(seg_list)

for i in range(0, num-1):

    threads.append(threading.Thread(target = getUserReviews, 
                                    args = (url_list, seg_list[i], seg_list[i+1])))
    threads[i].start()

# 等待所有子執行緒結束
for index in range(0, num-1):
    threads[index].join()

logging.info("Done")
print("Done")











Done


In [111]:
df = pd.DataFrame(all_reviews)

print(len(df))
df.sample(6)

711708


,product,user,user_rating,user_comment
629194,Edradour 10 Year,Samueljlutz,3.00,None
205593,Kentucky Owl 11 Year Kentucky Straight Rye (Ba...,beaversm,5.00,None
467760,The Ileach Peaty,lekiboy,4.00,I like how this one starts and delivers on the...
422404,Eagle Rare 10 Year Bourbon,Giza,4.50,None
496275,Nine Banded Small Batch Whiskey,BradFreeney,3.25,None
431732,Jack Daniel's Single Barrel Barrel Proof,gldngrhm,4.00,None


## Export the Data

In [112]:
#Setting Timestamp
date = "".join(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()).split(' ')[0].split('-'))

In [113]:
#Output as CSV
df.to_csv(f'distiller_user_reviews_{date}.csv')

In [114]:
#Convert to JSON
json_data = json.dumps(all_reviews, sort_keys=True, ensure_ascii=False, indent=4)

In [116]:
#Output as JSON
with open(f'distiller_user_reviews_{date}.json','w') as file:
    file.write(json_data)